In [1]:
# import tensorflow_datasets as tfds
import pickle
import re
import string
from multiprocessing import Pool

import contractions
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import tqdm
from keras.layers import LSTM, Dense, Dropout, Embedding, TextVectorization
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pandas import DataFrame, Series
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
from tqdm.notebook import tqdm_notebook

tqdm_notebook.pandas()

# tqdm.pandas()

2024-07-24 17:11:12.529246: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


About Sentiment140

This is the sentiment140 dataset.

It contains 1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 2 = neutral, 4 = positive) and they can be used to detect sentiment .

It contains the following 6 fields:

1. target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
2. ids: The id of the tweet ( 2087)
3. date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)
4. flag: The query (lyx). If there is no query, then this value is NO_QUERY.
5. user: the user that tweeted (robotickilldozr)
6. text: the text of the tweet (Lyx is cool)

Cargar datos desde los datasets de tensorflow

In [6]:
# df = tfds.load("sentiment140", data_dir="./input")

In [4]:
columns = ["target", "id", "date", "flag", "user", "text"]
df = pd.read_csv("./input/sentiment140.csv", encoding="ISO-8859-1", names=columns)

In [5]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Eliminacion de las columnas que no son necesarias

In [6]:
df.drop(["id", "date", "flag", "user"], axis="columns", inplace=True)
df.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


Como vemos no hay valores faltantes, pero si hay algunas filas duplicadas

In [7]:
def summary(df: DataFrame):
    print("Shape: ", df.shape)
    print("Duplicate rows: ", df.duplicated().sum())
    return pd.DataFrame(
        index=df.columns,
        data={
            "Unique": df.nunique().values,
            "Missing": df.isnull().sum().values,
            "Type": df.dtypes,
        },
    )

In [8]:
summary(df)

Shape:  (1600000, 2)
Duplicate rows:  16309


,Unique,Missing,Type
target,2,0,int64
text,1581466,0,object


Eliminacion de filas duplicadas

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
for i in range(100):
    print(df["text"][i])

@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!
@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds
my whole body feels itchy and like its on fire 
@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. 
@Kwesidei not the whole crew 
Need a hug 
@LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?
@Tatiana_K nope they didn't have it 
@twittera que me muera ? 
spring break in plain city... it's snowing 
I just re-pierced my ears 
@caregiving I couldn't bear to watch it.  And I thought the UA loss was embarrassing . . . . .
@octolinz16 It it counts, idk why I did either. you never talk to me anymore 
@smarrison i would've been the first, but i didn't have a gun.    not rea

Inicializacion de variables a usar en la limpieza de datos

In [11]:
spell = SpellChecker()
wln = WordNetLemmatizer()
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /home/jared/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jared/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
def correct_spellings(text: str) -> str:
    words = text.split()
    misspelled = spell.unknown(words)
    for i in range(len(words)):
        if words[i] in misspelled:
            word = spell.correction(words[i])
            if word:
                words[i] = word  # type: ignore
    return " ".join(words)

In [16]:
correct_spellings("corect is you runing")

'correct is you running'

In [17]:
def delete_stopwords(text: str):
    stop_words = stopwords.words("english")
    words = [word for word in text.split() if word not in stop_words]
    return " ".join(words)

In [18]:
delete_stopwords(
    "i would have been the first but i did not have a gun not really though zap spders jus a doucheclown"
)

'would first gun really though zap spders jus doucheclown'

In [19]:
def lemmatizer(text: str):
    words = [wln.lemmatize(word, pos="v") for word in text.split()]
    return " ".join(words)

In [20]:
lemmatizer(
    "is upset that he can't update his Facebook by texting it and might cry as a result  School today also. Blah!"
)

"be upset that he can't update his Facebook by texting it and might cry as a result School today also. Blah!"

In [12]:
def clean_text(text: str):
    if not text == "":
        text = text.lower()  # Convertir minusculas todo el texto
        text = re.sub(
            r"@[\S]+", "", text
        )  # Eliminar los nombres de usuarios con @ mencionados
        text = re.sub(
            r"((www\.[\S]+)|([https]+://[\S]+))", "", text
        )  # Eliminar las urls mencionadas
        text = re.sub(
            r"^\s+|\s+$|\s+(?=\s)", "", text
        )  # Eliminar espacios en blanco extras
        text = contractions.fix(text)  # type: ignore # Expandir las contracciones

        text = re.sub(
            "[%s]" % re.escape(string.punctuation), "", text
        )  # Eliminar signos de puntuacion
        # text = re.sub(r"\w*\d\w*", "", text)  # Eliminar numeros y palabras con numeros
        text = re.sub(r"[^A-Za-z\s]*", "", text)
        # text = correct_spellings(text)  # Corregir ortografia de palabras
        # text = delete_stopwords(text)  # Eliminar palabas comunes
        # text = lemmatizer(text)  # Convertir las palabras a su verbo base
    return text

In [16]:
def process(data_part):
    text = data_part["text"].apply(clean_text)
    return text

In [18]:
partitions = np.array_split(df, 16)
pool = Pool(16)

In [19]:
df_parts = pool.map(process, partitions)

In [20]:
clean_df = pd.concat(df_parts)

In [22]:
PATH = "./input/glove.840B.300d.txt"


embedding_dict = {}
with open(PATH, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split(" ")
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors

In [23]:
with open("./input/glove.840B.300d.pkl", "wb") as f:
    pickle.dump(embedding_dict, f)

In [21]:
with open("./input/glove.840B.300d.pkl", "rb") as f:
    embedding_dict = pickle.load(f)

In [22]:
import operator


def build_vocab(sentences: list) -> dict[str, float]:
    vocab = {}
    for text in sentences:
        for word in text.split():
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab


def check_coverage(
    vocab: dict[str, float], embeddings_index: dict[str, float]
) -> tuple[float, float, list[tuple]]:
    covered_words = {}
    oov = {}
    n_covered = n_oov = 0
    for word in vocab:
        try:
            covered_words[word] = embeddings_index[word]
            n_covered += vocab[word]
        except KeyError:
            oov[word] = vocab[word]
            n_oov += vocab[word]
    vocab_coverage = len(covered_words) / (len(vocab))
    text_coverage = n_covered / (n_covered + n_oov)
    sorted_oov = sorted(oov.items(), key=operator.itemgetter(1))[::-1]
    return (vocab_coverage, text_coverage, sorted_oov)

In [23]:
vocab = build_vocab(list(clean_df))

In [24]:
vocab

{'awww': 4980,
 'that': 172038,
 'is': 383256,
 'a': 376520,
 'bummer': 1442,
 'you': 364888,
 'shoulda': 348,
 'got': 69383,
 'david': 2267,
 'carr': 75,
 'of': 189222,
 'third': 814,
 'day': 84050,
 'to': 614198,
 'do': 136474,
 'it': 298478,
 'd': 6757,
 'upset': 2641,
 'he': 37483,
 'cannot': 64692,
 'update': 4243,
 'his': 15376,
 'facebook': 4196,
 'by': 30105,
 'texting': 727,
 'and': 297529,
 'might': 9620,
 'cry': 4395,
 'as': 40616,
 'result': 594,
 'school': 19597,
 'today': 64280,
 'also': 10252,
 'blah': 1471,
 'i': 984284,
 'dived': 3,
 'many': 8816,
 'times': 7664,
 'for': 214795,
 'the': 523491,
 'ball': 1272,
 'managed': 976,
 'save': 2190,
 'rest': 5203,
 'go': 72480,
 'out': 80092,
 'bounds': 19,
 'my': 311949,
 'whole': 5949,
 'body': 2467,
 'feels': 5120,
 'itchy': 440,
 'like': 77539,
 'its': 43693,
 'on': 166169,
 'fire': 1503,
 'no': 73713,
 'not': 281406,
 'behaving': 68,
 'at': 110561,
 'all': 82820,
 'am': 240021,
 'mad': 3793,
 'why': 27640,
 'here': 36719,


In [25]:
len(vocab)

414983

In [26]:
vocab_coverage, text_coverage, oov = check_coverage(vocab, embedding_dict)

In [27]:
print("Found embeddings for {:.2%} of vocab".format(vocab_coverage))
print("Found embeddings for  {:.2%} of all text".format(text_coverage))

Found embeddings for 31.90% of vocab
Found embeddings for  97.93% of all text


In [28]:
oov[:5]

[('quoti', 1604),
 ('gtlt', 813),
 ('quotyou', 628),
 ('youquot', 600),
 ('iranelection', 487)]

In [29]:
spell = SpellChecker(distance=1)

In [30]:
words_df = pd.DataFrame({"word": list(dict(oov).keys())})

In [31]:
def correct_spellings(word: str):
    correction = spell.correction(word)
    if correction:
        return correction
    return word

In [32]:
def task(data_part):
    text = data_part["word"].apply(correct_spellings)
    return text

In [33]:
partitions = np.array_split(words_df, 16)
pool = Pool(16)

/home/jared/micromamba/envs/delfin2024/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [34]:
df_parts = pool.map(task, partitions)

In [35]:
words_df = pd.concat(df_parts)

In [36]:
new_words = {}
for i in range(len(oov)):
    if oov[i][0] != words_df.loc[i]:
        new_words[oov[i][0]] = words_df.loc[i]

In [37]:
len(new_words)

46186

In [38]:
def correct_word(text: str):
    words = text.split()
    for i in range(len(words)):
        try:
            words[i] = new_words[words[i]]
        except KeyError:
            pass
    return " ".join(words)

In [39]:
clean_df = clean_df.apply(correct_word)  # type: ignore

In [40]:
vocab = build_vocab(list(clean_df))

In [41]:
len(vocab)

373089

In [42]:
vocab_coverage, text_coverage, oov = check_coverage(vocab, embedding_dict)
print("Found embeddings for {:.2%} of vocab".format(vocab_coverage))
print("Found embeddings for  {:.2%} of all text".format(text_coverage))

Found embeddings for 36.35% of vocab
Found embeddings for  98.30% of all text


In [43]:
max_tokens = len(vocab) - len(oov)

In [44]:
count = 0
for text in clean_df:
    if text == "":
        count += 1
count

3572

In [45]:
len(clean_df)

1583691

In [46]:
len(df)

1583691

In [3]:
tokenizer = TextVectorization(standardize=None)

2024-07-24 17:12:07.434397: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 17:12:07.608835: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 17:12:07.609069: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [47]:
tokenizer = TextVectorization(standardize=None, max_tokens=max_tokens)
tokenizer.adapt(clean_df)
vectorized_texts = tokenizer(clean_df)
word_index = tokenizer.get_vocabulary()

In [48]:
max_tokens

135620

In [49]:
embedding_dim = 300  # Dimensionalidad de los embeddings de GloVe
count = 0
# Crear una matriz de embeddings
embedding_matrix = np.zeros((len(word_index), embedding_dim))
for i, word in enumerate(word_index):
    embedding_vector = embedding_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        count += 1

print(f"Matriz de embeddings: {embedding_matrix.shape}")

Matriz de embeddings: (135620, 300)


In [50]:
count

90667

In [56]:
X = np.array(vectorized_texts)
y = df["target"].replace(4, 1).values

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [58]:
vectorized_texts.shape[1]

40

In [59]:
max_tokens

135620

In [60]:
embedding_matrix.shape

(135620, 300)

In [51]:
# type: ignore
model = tf.keras.Sequential(
    [
        Embedding(max_tokens, embedding_dim, weights=[embedding_matrix]),
        Dropout(0.5),
        LSTM(100, dropout=0.2, recurrent_dropout=0.2),
        Dense(1, activation="sigmoid"),
    ],
)

In [58]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │    40,686,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,686,000 (155.20 MB)

 Trainable params: 40,686,000 (155.20 MB)

 Non-trainable params: 0 (0.00 B)

In [59]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [60]:
callbacks = tf.keras.callbacks.EarlyStopping(
    monitor="val_acc", min_delta=1e-4, patience=5
)

In [62]:
history = model.fit(
    X_train,
    y_train,
    batch_size=1024,
    epochs=8,
    validation_split=0.1,
    verbose=1,
)

Epoch 1/8
1114/1114 ━━━━━━━━━━━━━━━━━━━━ 51s 45ms/step - accuracy: 0.8099 - loss: 0.4146 - val_accuracy: 0.8200 - val_loss: 0.3957
Epoch 2/8
1114/1114 ━━━━━━━━━━━━━━━━━━━━ 51s 46ms/step - accuracy: 0.8222 - loss: 0.3927 - val_accuracy: 0.8245 - val_loss: 0.3893
Epoch 3/8
1114/1114 ━━━━━━━━━━━━━━━━━━━━ 51s 46ms/step - accuracy: 0.8312 - loss: 0.3753 - val_accuracy: 0.8245 - val_loss: 0.3872
Epoch 4/8
1114/1114 ━━━━━━━━━━━━━━━━━━━━ 51s 46ms/step - accuracy: 0.8381 - loss: 0.3629 - val_accuracy: 0.8262 - val_loss: 0.3865
Epoch 5/8
1114/1114 ━━━━━━━━━━━━━━━━━━━━ 52s 46ms/step - accuracy: 0.8437 - loss: 0.3518 - val_accuracy: 0.8251 - val_loss: 0.3893
Epoch 6/8
1114/1114 ━━━━━━━━━━━━━━━━━━━━ 52s 46ms/step - accuracy: 0.8490 - loss: 0.3416 - val_accuracy: 0.8251 - val_loss: 0.3930
Epoch 7/8
1114/1114 ━━━━━━━━━━━━━━━━━━━━ 52s 46ms/step - accuracy: 0.8537 - loss: 0.3317 - val_accuracy: 0.8256 - val_loss: 0.4002
Epoch 8/8
1114/1114 ━━━━━━━━━━━━━━━━━━━━ 52s 47ms/step - accuracy: 0.8572 - loss: 0

In [64]:
model.evaluate(X_test, y_test, batch_size=1024)

310/310 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8242 - loss: 0.4047


[0.4053652286529541, 0.824877917766571]

In [87]:
def predict(text: str):
    return "Negativo" if model.predict(tokenizer([text]))[0] <= 0.49 else "Positivo"

In [68]:
tokenizer(["hello name"])

<tf.Tensor: shape=(1, 2), dtype=int64, numpy=array([[425, 417]])>

In [89]:
predict("i can not run")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


'Negativo'

In [90]:
predict("I hate the rain")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


'Negativo'

In [91]:
predict("I love the music")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


'Positivo'

In [95]:
predict("I like pizza")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


'Positivo'